In [13]:
import gzip
import json
import ast
import pandas as pd
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer



# Ruta al archivo JSON comprimido
archivo_gz_user = r"C:\Users\veram\OneDrive\Escritorio\proyecto octubre\user_reviews.json.gz"

# Lectura del archivo JSON comprimido
data_user = []

try:
    with gzip.open(archivo_gz_user, 'rt', encoding='utf-8') as f:
        for line_number, line in enumerate(f, start=1):
            try:
                # Usamos ast.literal_eval para evaluar la línea como un diccionario de Python
                data_object = ast.literal_eval(line)
                data_user.append(data_object)  # Añadimos el objeto a la lista
            except (SyntaxError, ValueError) as e:
                print(f"Error al evaluar la línea {line_number}: {e}")
                print(f"Línea problemática: {line[:100]}...")  # Imprimimos parte de la línea problemática
                continue
    
    print(f"Se han cargado {len(data_user)} objetos correctamente.")
    print(data_user[:5])  # Mostramos los primeros 5 objetos
except FileNotFoundError:
    print("El archivo no se encontró. Verifica la ruta.")
except Exception as e:
    print(f"Ocurrió un error al cargar los datos: {e}")



# Convertimos los datos a un dataframe
df_user = pd.DataFrame(data_user)

Se han cargado 25799 objetos correctamente.
[{'user_id': '76561197970982479', 'user_url': 'http://steamcommunity.com/profiles/76561197970982479', 'reviews': [{'funny': '', 'posted': 'Posted November 5, 2011.', 'last_edited': '', 'item_id': '1250', 'helpful': 'No ratings yet', 'recommend': True, 'review': 'Simple yet with great replayability. In my opinion does "zombie" hordes and team work better than left 4 dead plus has a global leveling system. Alot of down to earth "zombie" splattering fun for the whole family. Amazed this sort of FPS is so rare.'}, {'funny': '', 'posted': 'Posted July 15, 2011.', 'last_edited': '', 'item_id': '22200', 'helpful': 'No ratings yet', 'recommend': True, 'review': "It's unique and worth a playthrough."}, {'funny': '', 'posted': 'Posted April 21, 2011.', 'last_edited': '', 'item_id': '43110', 'helpful': 'No ratings yet', 'recommend': True, 'review': 'Great atmosphere. The gunplay can be a bit chunky at times but at the end of the day this game is definit

In [14]:
# aplicamos "explode" a la columna 'reviews'
df_exploded = df_user.explode('reviews').reset_index(drop=True)

# Convertimos los diccionarios de la columna 'reviews' en columnas
df_expanded = pd.json_normalize(df_exploded['reviews'])
df_reviews_user = pd.concat([df_exploded.drop(columns=['reviews']), df_expanded], axis=1)


In [15]:
# Enlistamos y después BORRAMOS las columnas que consideramos INNECESARIAS para el proyecto 
columnas_a_borrar = ['user_url','helpful', 'funny','last_edited']

df_reviews_user.drop(columns=columnas_a_borrar, inplace=True)

In [17]:
# Encontramos las filas donde TODOS los valores son NULOS
filas_nulas = df_reviews_user.isnull().all(axis=1)

# Obtenemos los índices donde TODAS las columnas son NULAS
indices_vacios = df_reviews_user.index[filas_nulas]

# Verificamos el índice máximo hasta donde TODAS las filas están vacías
if not indices_vacios.empty:
    max_indices_vacios =indices_vacios.max()
    print(f"El índice máximo hasta donde todas las filas están vacías es: {max_indices_vacios}")
else:
    print("No hay filas completamente vacías en el DataFrame.")

No hay filas completamente vacías en el DataFrame.


In [18]:
#Contamos los valores faltantes en las columnas review, posted, item_id, recommend,user_id
faltantes_reviews = df_reviews_user['review'].isnull().sum()
print(f"Valores faltantes en la columna 'review': {faltantes_reviews}")

faltantes_posted = df_reviews_user['posted'].isna().sum()
print(f"Número de valores faltantes en 'posted': {faltantes_posted}")

faltantes_item_id = df_reviews_user['item_id'].isna().sum()
print(f"Número de valores faltantes en 'item_id': {faltantes_item_id}")

faltantes_recommend = df_reviews_user['recommend'].isna().sum()
print(f"Número de valores faltantes en 'recommend': {faltantes_recommend}")

faltantes_user_id = df_reviews_user['user_id'].isna().sum()
print(f"Número de valores faltantes en 'user_id': {faltantes_user_id}")




Valores faltantes en la columna 'review': 28
Número de valores faltantes en 'posted': 28
Número de valores faltantes en 'item_id': 28
Número de valores faltantes en 'recommend': 28
Número de valores faltantes en 'user_id': 0


In [19]:
#Eliminamos la palabra "Posted" y el punto al final de cada fecha
df_reviews_user['posted'] = df_reviews_user['posted'].str.replace('Posted', '', regex=False).str.replace('.', '', regex=False).str.strip()

In [20]:

#Extraemos solo el año utilizando expresiones regulares
df_reviews_user['year_posted'] = df_reviews_user['posted'].str.extract(r'(\d{4})')



In [21]:
# Contamos los valores faltantes en la columna year_posted
faltantes_year_posted = df_reviews_user['year_posted'].isna().sum()
print(f"Número de valores faltantes en 'year_posted': {faltantes_year_posted}")

Número de valores faltantes en 'year_posted': 10147


In [22]:
# Borramos filas con valores faltantes
df_reviews_user_cleaned = df_reviews_user.dropna()

In [23]:
# Borramos la columna posted ya que creamos la columna que contiene el año
df_reviews_user_cleaned.drop(columns=['posted'], inplace=True)

C:\Users\veram\AppData\Local\Temp\ipykernel_19248\762017775.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_reviews_user_cleaned.drop(columns=['posted'], inplace=True)


In [24]:
#Reordenamos las columnas del dataframe
nuevo_orden_columnas = [
    'user_id',
    'item_id',
    'review',
    'year_posted',
    'recommend',  
]

df_reviews_users_orden = df_reviews_user_cleaned[nuevo_orden_columnas]
#verificamos el cambio
df_reviews_users_orden

,user_id,item_id,review,year_posted,recommend
0,76561197970982479,1250,Simple yet with great replayability. In my opi...,2011,True
1,76561197970982479,22200,It's unique and worth a playthrough.,2011,True
2,76561197970982479,43110,Great atmosphere. The gunplay can be a bit chu...,2011,True
3,js41637,251610,I know what you think when you see this title ...,2014,True
4,js41637,227300,For a simple (it's actually not all that simpl...,2013,True
...,...,...,...,...,...
59280,wayfeng,730,its FUNNNNNNNN,2015,True
59283,76561198251004808,253980,Awesome fantasy game if you don't mind the gra...,2015,True
59293,72947282842,730,Prettyy Mad Game,2015,True
59295,ApxLGhost,730,AMAZING GAME 10/10,2015,True


In [27]:
# Creamos una instancia de SentimentIntensityAnalyzer
sia = SentimentIntensityAnalyzer()

#Creamos la función para determinar el sentimiento
def get_sentiment(text):
    # Verificamos si el texto es NaN o está vacío
    if pd.isna(text) or text.strip() == '':
        return 1  # Valor 1 si no hay reseña
    score = sia.polarity_scores(text)['compound']
    if score < 0:
        return 0  # Malo
    elif score == 0:
        return 1  # Neutral
    else:
        return 2  # Positivo
    

# Aplicamos la función al dataframe
df_reviews_users_orden['sentiment_analysis'] = df_reviews_users_orden['review'].apply(get_sentiment)
#verificamos 
print(df_reviews_users_orden)


                 user_id item_id  \
0      76561197970982479    1250   
1      76561197970982479   22200   
2      76561197970982479   43110   
3                js41637  251610   
4                js41637  227300   
...                  ...     ...   
59280            wayfeng     730   
59283  76561198251004808  253980   
59293        72947282842     730   
59295          ApxLGhost     730   
59304  76561198267374962  369200   

                                                  review year_posted  \
0      Simple yet with great replayability. In my opi...        2011   
1                   It's unique and worth a playthrough.        2011   
2      Great atmosphere. The gunplay can be a bit chu...        2011   
3      I know what you think when you see this title ...        2014   
4      For a simple (it's actually not all that simpl...        2013   
...                                                  ...         ...   
59280                                     its FUNNNNNNNN       

C:\Users\veram\AppData\Local\Temp\ipykernel_19248\3994888927.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_reviews_users_orden['sentiment_analysis'] = df_reviews_users_orden['review'].apply(get_sentiment)


In [28]:
# Borramos la columna REVIEW para aligerar el dataset
df_reviews_users_orden.drop(columns=['review'], inplace=True)

# Guardamos el dataframe en un archivo Parquet
df_reviews_users_orden.to_parquet('df_review_user.parquet', index=False)
#verificamos
print("El archivo Parquet ha sido guardado exitosamente.")

El archivo Parquet ha sido guardado exitosamente.
